In [ ]:

%run includes/7-cloudfunctions-setup.ipynb

In [ ]:

# cloud functions
locations_queries = [
    f"""
    SELECT locationId,
    '{project}' as project
    FROM google.cloudfunctions.locations WHERE projectsId = '{project}'
    """
    for project in all_projects
]

locations_df = run_stackql_queries(locations_queries)
# locations = locations_df['locationId'].unique().tolist()
locations = ['australia-southeast1']
cf_projects = locations_df['project'].unique().tolist()

functions_queries = [
    f"""
    SELECT 
    SPLIT_PART(name, '/', 6) as name,
    description,
    url,
    '{project}' as project,
    '{location}' as location,
    JSON_EXTRACT(buildConfig, '$.runtime') as runtime,
    JSON_EXTRACT(serviceConfig, '$.availableMemory') as availableMemory,
    JSON_EXTRACT(serviceConfig, '$.revision') as revision,
    JSON_EXTRACT(serviceConfig, '$.timeoutSeconds') as timeoutSeconds,
    JSON_EXTRACT(serviceConfig, '$.serviceAccountEmail') as serviceAccountEmail,
    JSON_EXTRACT(serviceConfig, '$.ingressSettings') as ingressSettings,
    JSON_EXTRACT(serviceConfig, '$.maxInstanceCount') as maxInstanceCount,
    JSON_EXTRACT(serviceConfig, '$.maxInstanceRequestConcurrency') as maxInstanceRequestConcurrency,
    JSON_EXTRACT(serviceConfig, '$.environmentVariables') as environmentVariables,
    eventTrigger,
    state,
    stateMessages,
    environment,
    kmsKeyName,
    satisfiesPzs,
    labels
    FROM google.cloudfunctions.functions WHERE projectsId = '{project}' AND locationsId = '{location}'
    """
    for project in cf_projects
    for location in locations
]
functions_df = run_stackql_queries(functions_queries)

## Function Summary

In [ ]:

# Plot functions by project
plot_functions_by_project(functions_df)
# Plot functions by location, runtime and available memory
plot_all_side_by_side(functions_df)

## Function Details

In [ ]:

# function details
functions_df